In [2]:
from pycromanager import Core, Studio
from smartpath_libraries.sp_acquisition import SPAcquisition

In [7]:
def init_pycromanager():
    from pycromanager import Core, Studio
    core = Core()
    studio = Studio()
    core.set_timeout_ms(20000)
    return core, studio
core,studio = init_pycromanager()

In [3]:
## config : default config only:  changed will be saved to expt folder
import yaml
## read image: background correction
from skimage import io as skio
# pos list
import pickle

In [4]:
## Config Loading
config = yaml.load(open("config.yaml", "r"), Loader=yaml.FullLoader)
config = {**config['user_config'], **config['model_config'], **config['hard_config']}
lsm_pixel_size_base = config['pixel-size-shg-base']
bf_4x_pixel_size_base = config['pixel-size-bf-4x']
bf_20x_pixel_size_base = config['pixel-size-bf-20x']
camera_resolution_base = config['camera-resolution']

In [5]:
## User configuration block
save_path = 'data/acquisition' # do not change
acq_name = 'fail_test2' # current session name No Spaces in name! Use - or _ No `ome` in name!
slide_box = [5000, 9000, 9000, 14000] # [bottom left corner, and top right corner]
brightfield_4x_background_fname = 'data/presets/BG_4x.tiff' # give a default 4x background image
brightfield_20x_background_fname = 'data/presets/BG_20x.tiff' # give a default 20x background image

In [8]:
sp_acq = SPAcquisition(
    config=config,
    mmcore=core, 
    mmstudio=studio,
    bf_4x_bg=skio.imread(brightfield_4x_background_fname), 
    bf_20x_bg=skio.imread(brightfield_20x_background_fname), 
    )

Load LSM presets.


In [9]:
sp_acq.update_slide_box(slide_box)
position_list_4x = sp_acq.generate_grid(mag='4x', overlap=50)
sp_acq.position_list_4x = position_list_4x

In [10]:
position_list_4x

array([[[ 5000.  ,  9000.  ],
        [ 6482.91,  9000.  ]],

       [[ 6482.91, 10093.95],
        [ 5000.  , 10093.95]],

       [[ 5000.  , 11187.9 ],
        [ 6482.91, 11187.9 ]],

       [[ 6482.91, 12281.85],
        [ 5000.  , 12281.85]]])

In [54]:
position_list_4x.ndim

3

In [11]:
acq_name_4x = acq_name + '-4x-bf' 
sp_acq.switch_objective(mag='4x')
sp_acq.switch_mod(mod='bf')

Imaging objective set as 4x
Imaging objective set as 4x
Imaging mode set as Brightfield


In [12]:
sp_acq.update_focus_presets(mag='4x', mod='bf') # update focus preset
sp_acq.config['autofocus-speed'] = 4 # default is 4 ## `1-6`, the larger the faster, but potentially worse autofocusing resuls.

In [13]:
core.set_shutter_open(True)
core.set_auto_shutter(False)
core.set_shutter_open(True)

In [14]:
results_4x = sp_acq.whole_slide_bf_scan(
    save_path, 
    acq_name_4x, 
    position_list_4x.reshape(position_list_4x.shape[0]*position_list_4x.shape[1], -1), 
    mag='4x',
    focus_dive=True,
    estimate_background=False,
    )

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:16<00:00,  2.03s/it]

 8/8 tiles done

In [19]:
stitch_folder = r'C:\Users\lociuser\Codes\smart-wsi-scanner\data\acquisition\fail_test2-4x-bf_1'
position_list = position_list_4x
pixel_size = config['pixel-size-bf-4x']

In [17]:
import os

In [28]:
position_list.shape

(4, 2, 2)

In [37]:
import numpy as np
position_list = np.reshape(position_list,[4*2,2])

In [38]:
for pos in range(position_list.shape[0]):
    #x = int(position_list[pos][0] / pixel_size)
    #y = int(position_list[pos][1] / pixel_size)
    #print('{}.tiff; ; ({}, {})'.format(pos, x, y), file=text_file)
    print(position_list[pos])

[5000. 9000.]
[6482.91 9000.  ]
[ 6482.91 10093.95]
[ 5000.   10093.95]
[ 5000.  11187.9]
[ 6482.91 11187.9 ]
[ 6482.91 12281.85]
[ 5000.   12281.85]


In [40]:
import glob
import pathlib
import shutil

In [46]:
for fn in glob.glob(r'C:\Users\lociuser\Codes\smart-wsi-scanner\data\acquisition\fail_test2-4x-bf_1\*.tif'):
    fname = pathlib.Path(fn).name
    nfname = fname.split('-')[0]+'.tif'
    #print(nfname)
    nfn = fn.replace(fname,nfname)
    shutil.copy(fn,nfn)

In [47]:
with open(os.path.join(stitch_folder, 'TileConfiguration.txt'), 'w') as text_file:
    print('dim = {}'.format(2), file=text_file)
    for pos in range(position_list.shape[0]):
        x = int(position_list[pos][0] / pixel_size)
        y = int(position_list[pos][1] / pixel_size)
        print('{}.tif; ; ({}, {})'.format(pos, x, y), file=text_file)

In [50]:
image_list = glob.glob(r'C:\Users\lociuser\Codes\smart-wsi-scanner\data\acquisition\fail_test2-4x-bf_1\raw\*.tif')

In [52]:
from skimage import io,  img_as_uint, transform

In [53]:
for pos in range(len(image_list)):
    
    img = io.imread(image_list[pos])

    correction= False
    rotate=False
    flip_y=True
    flip_x=False
    
    if correction is True and background_image is not None:
        img = white_balance(img, background_image)
        img = flat_field(img, bg_img)
    
    if rotate is not None:
        img = transform.rotate(img, rotate)
    
    if flip_y:
        img = img[::-1, :]
    
    if flip_x:
        img = img[:, ::-1]
    io.imsave(stitch_folder+'/{}.tif'.format(pos), img_as_uint(img), check_contrast=False)